In [3]:
import os, json, uuid, requests
from datetime import datetime, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju

load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"
QR_API_URL = os.getenv(
    "QR_API_URL",
    "https://faas-sgp1-18bc02ac.doserverless.co/api/v1/web/fn-86217a9f-1135-4904-b49a-fe070d4e10c7/forms/generate-qr-code",
)

if not uri or not database_name:
    raise RuntimeError("Set MONGODB_URI and MONGODB_DATABASE in your .env")

client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

def jprint(obj):
    print(bsonju.dumps(obj, indent=2, ensure_ascii=False))

def _generate_current_form_qr_code():
    try:
        resp = requests.post(QR_API_URL, json={}, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        code = str(data.get("current_form_qr_code") or "").strip()
        if code:
            return code
    except Exception as e:
        print(f"⚠️ QR code generation failed: {e}")
    return datetime.now(timezone.utc).strftime("%Y%m%d")

latest_empty = col.find_one(
    {
        "is_empty": True,
        "$or": [{"items": {"$exists": False}}, {"items": {"$size": 0}}],
    },
    sort=[("created", -1)],
)

if latest_empty:
    print("✅ Reusing latest EMPTY form (no new document created).")
    jprint(latest_empty)
else:
    latest_doc = col.find_one(sort=[("created", -1)]) or {}
    prev_qr       = latest_doc.get("current_form_qr_code", "")
    prev_tin      = latest_doc.get("tin", "")
    prev_location = latest_doc.get("location", "")

    now_utc = datetime.now(timezone.utc)
    date_midnight_utc = datetime(year=now_utc.year, month=now_utc.month, day=now_utc.day, tzinfo=timezone.utc)
    current_qr = _generate_current_form_qr_code()

    new_doc = {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_qr,
        "current_form_qr_code": current_qr,
        "tin": prev_tin,
        "location": prev_location,
        "created": now_utc,
        "date": date_midnight_utc,
        "items": [],
        "employees": [
            {"role": "cashier"},
            {"role": "recorder"},
        ],  # roles only; no employee_number and name
        "is_empty": True,
    }

    res = col.insert_one(new_doc)
    new_doc["_id"] = res.inserted_id

    print("✅ Created new EMPTY form (employees include roles only).")
    jprint(new_doc)


✅ Created new EMPTY form (employees include roles only).
{
  "id": "1d020019-2c43-4bd8-8a52-caac4aeb9251",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "52741246",
  "tin": "123-456-789",
  "location": "JEF Gas Station – Sikatuna Branch",
  "created": {
    "$date": 1761097322765
  },
  "date": {
    "$date": 1761091200000
  },
  "items": [],
  "employees": [
    {
      "role": "cashier"
    },
    {
      "role": "recorder"
    }
  ],
  "is_empty": true,
  "_id": {
    "$oid": "68f8366d800ddf5dffd44541"
  }
}
